# 나이브 베이즈 정리 모델로 발라드 노래 감정 분류
---

In [10]:
# 
from bayes import BayesianFilter
import pandas as pd


In [11]:
# 학습시킬 데이터 셋 불러오기
train_ballad80 = pd.read_csv("../../Data/train_data.csv")
train_ballad80

,Unnamed: 0,가수명,노래제목,발매일,가사,좋아요수,감정
0,0,양다일,Darling,2020-11-23,Oh darling 처음 널 만나면 수줍은 대화들로 사랑에 빠질 거야 My darl...,10907,happy
1,1,장범준,잠이 오질 않네요,2020-10-24,당신은 날 설레게 만들어 조용한 내 마음 자꾸만 춤추게 해 얼마나 얼마나 날 떨리게...,159018,happy
2,2,백아연,어디라도,2020-11-18,우연의 다른 이름은 운명이란 걸 배웠죠 우리 사랑의 끝이 없는 걸 이제서야 알았죠 ...,1622,happy
3,3,김제훈,Love Again,2017-02-06,언제부터였을까 따뜻한 휴식을 찾고 있을 때 찾고 있을 때 그때부터였을까 맘이 포근해...,980,happy
4,4,김제훈,잠잠,2020-03-17,오랜만에 거길 가봤어 아무도 없는 둘만이 알던 곳 함께 걷던 거리에 서서 아무 말도...,548,happy
...,...,...,...,...,...,...,...
75,75,#안녕,너의 번호를 누르고 (Prod. 영화처럼),2019-11-30,우연히 너를 만나서 너의 옆자리에 앉아 그렇게 우린 친해졌어 짧은 시간에 그렇게 가...,109930,indifference
76,76,부활,Never Ending Story,2002-09-01,손 닿을 수 없는 저기 어딘가 오늘도 난 숨쉬고 있지만 너와 머물던 작은 의자 위에...,70543,indifference
77,77,Chocolat.D (쇼콜라디),그날에 우리,2016-02-05,그날에 우리 예감했었지 끊어질 것 같은 우리의 사랑 차마 말하지 못했지 마지막 그 ...,61,indifference
78,78,정승수,되돌릴 수 없는 이별을 해,2021-01-10,혼자 집에 걸어가는 길이 어색해 우리 예전처럼 함께 걷던 그 날이 무색해질 만큼 변...,878,indifference


---
### 감정분류가 되어있는 발라드 80 곡의 정보

In [12]:
train_ballad80.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  80 non-null     int64 
 1   가수명         80 non-null     object
 2   노래제목        80 non-null     object
 3   발매일         80 non-null     object
 4   가사          80 non-null     object
 5   좋아요수        80 non-null     int64 
 6   감정          80 non-null     object
dtypes: int64(2), object(5)
memory usage: 4.5+ KB


In [13]:
# 첫번째 노래 추출하기
train_ballad80.iloc[0]

Unnamed: 0                                                    0
가수명                                                         양다일
노래제목                                                    Darling
발매일                                                  2020-11-23
가사            Oh darling 처음 널 만나면 수줍은 대화들로 사랑에 빠질 거야 My darl...
좋아요수                                                      10907
감정                                                        happy
Name: 0, dtype: object

In [14]:
# train_ballad80의 첫번째 노래의 가사
str(train_ballad80.iloc[0].가사)

'Oh darling 처음 널 만나면 수줍은 대화들로 사랑에 빠질 거야 My darling 아마도 우린 손을 마주 잡고서 도시를 거닐 거야 비라도 오면 골목 어귀에 발을 멈추고서 바라보다가 입을 맞출지도 몰라 어둠이 오면 밝게 빛나는 별빛들 아래서 음악에 취해 서로에게 기대어 Darling My darling 멈춰버린 시간 속에서 우리 둘이서 영원토록 Darling My darling 함께 할 거야 너의 곁에서 눈을 감을 때까지 Oh darling 잊지 못할 거야 서로를 품에 안고 온기를 느끼던 날 My darling 따스할 거야 함께 잠이 들 때면 비추는 햇살들이 가끔씩 우린 아쉬울 거야 다른 모습들에 말을 멈추고 어색한 침묵이 올 때면 서운한 마음에 다투다 보면 오직 둘만 아는 그런 말들로 가벼운 미소가 될 거야 Darling My darling 멈춰버린 시간 속에서 우리 둘이서 영원토록 Darling My darling 함께 할 거야 너의 곁에서 눈을 감을 때까지 널 처음 마주한 순간 알 수 있었어 나를 보며 지었던 그 미소 그게 내 전부란 걸 Darling My darling 설레이는 내 마음을 담아 너에게 고백할게 영원토록 Darling My darling 나를 안아줘 너의 곁에서 눈을 감을 때까지 눈을 감을 때까지 눈을 감을 때까지 '

In [15]:
# train_ballad80의 첫번째 노래의 감정
str(train_ballad80.iloc[0].감정)

'happy'

---
### 나이브베이즈 정리 모델 불러와 학습시키기

In [16]:
bf = BayesianFilter()

In [17]:
# for 문을 이용하여 텍스트 학습 시키기
for i in range(len(train_ballad80)):
    bf.fit(str(train_ballad80.iloc[i].가사),str(train_ballad80.iloc[i].감정))

In [18]:
# 예측 해보기
pre, scorelist = bf.predict("오늘 한잔할까요 늘 만나던 그곳에서 찬바람이 불면 생각이 나던 골목집에서 추억 한잔할까요 긴 하루의 끝에서 아름다웠던 우리의 시절 다시 돌아갈까요 어떤 누구보다 날 사랑했던 그 사람 한잔하고 싶을 때마다 떠오르는 그리움 혼자서 마시는 술에 홀로 남겨진 슬픔에 잊으려고 하면 할수록 더 그대가 보고 싶어요 나 많이 취한 것처럼 우는 것처럼 보여도 괜찮아요 그대 온다면 못 알아볼 리가 없죠 언제인지 몰라도 술 버릇이 생겼네요 테이블 위에다 흘린 술로 난 그대를 그리죠 오늘만은 내 잔을 뺏어가면 안 돼요 못 이기는 건 술이 아니라 아픈 이별이니까 돌아와야 하는 그대는 안 돌아오고 그대 떠난 그때 겨울만 혼자 돌아왔네요 혼자서 마시는 술에 홀로 남겨진 슬픔에 잊으려고 하면 할수록 더 그대가 보고 싶어요 나 많이 취한 것처럼 우는 것처럼 보여도 괜찮아요 그대 온다면 못 알아볼 리가 없죠 이렇게 있다간 나도 모르게 소리 내서 울 것 같아서 나 일어날래요 나 그만 갈래요 이제 자꾸 사람들이 쳐다보니까 나 항상 그대뿐이죠 그래 난 그대뿐이죠 헤어지고 나서 이제서야 그대란 걸 알아요 나 많이 취한 것처럼 우는 것처럼 보여도 내일에도 이 곳에 와서 그대를 기다릴게요")
print("결과 =", pre)
print(scorelist)

결과 = indifference
[('happy', -1186.060637858243), ('sad', -1156.540277567589), ('regret', -1148.2135705672147), ('indifference', -1147.8366858218412)]


In [19]:
# 예측 해보기
pre, scorelist = bf.predict("그댄 내가 바라던 Romance 오랫동안 헤맨듯해 꿈만 같은 그대 품에 Deep inside Hmm 이토록 많이 사랑하게 된 이유 저 파도처럼 내게 밀려왔었지 우린 좀 달라도 어딘가 특별해 그댄 내게 전부가 된 거죠 Oh oh 그댄 내가 바라던 Romance 오랫동안 헤맨듯해 꿈만 같은 그대 품에 Deep inside Hmm 그렇게 우린 서로에게 기대어 그 누구에게도 못한 이야기까지도 커다란 그 품속에 잠들 때면 모든 게 편해진 거죠 우린 서로의 밤이 되어요 Oh oh 그댄 내가 바라던 Romance 오랫동안 헤맨듯해 꿈만 같은 그대 품에 Deep inside Woo 그대와 난 내일을 그려가요 영원히 둘이서 사랑이라 부를게요 My romance ")
print("결과 =", pre)
print(scorelist)

결과 = sad
[('happy', -698.3833177636121), ('sad', -689.1805459533044), ('regret', -697.0318204826659), ('indifference', -690.1727000813985)]


In [20]:
# 예측 해보기
pre, scorelist = bf.predict("다신 사랑하지 않아 보고 싶지 않아 나를 떠나간 너를 내가 싫다고 했잖아 헤어지기 싫다고 옆에 있어달라고 눈물이 멈추질 않아 이렇게 끝이라면 다신 널 볼 수 없으니까 우리 사랑했던 추억을 아직 잊지 말아요 다시 돌아갈 수 없다며 단정 지었던 네가 너무 그립고 그리워서 보고 싶어 눈물이 나서 그대 생각하며 이렇게 조금만 더 아플게요 그대 행복해야 해요 아프지도 말고 내가 없었단 듯이 사실 아파했음 좋겠어 내 생각을 하면서 매일 밤마다 날 그리워해 주길 우리 사랑했던 추억을 아직 잊지 말아요 다시 돌아갈 수 없다며 단정 지었던 네가 너무 그립고 그리워서 보고 싶어 눈물이 나서 그대 생각하며 이렇게 조금만 더 아플게요 너와 걷던 길을 혼자 걸어요 그대 추억을 두고 오려 해 미안하다고 내가 보고 싶었다고 말 한마디면 난 충분한데 우리 사랑했던 추억을 아직 잊지 말아요 다시 돌아갈 수 없다며 단정 지었던 네가 너무 그립고 그리워서 눈물이 멈추질 않아서 그댈 생각하며 이렇게 나 조금만 더 아플게요 ")
print("결과 =", pre)
print(scorelist)

결과 = regret
[('happy', -952.5122474792649), ('sad', -886.5808754418125), ('regret', -875.9330093518503), ('indifference', -913.2004275417893)]


---
### 발라드 2000곡(베이즈 정리 모델 곡 제외)를 학습시키기

In [21]:
# 학습시킬 데이터 셋 불러오기
train_ballad2000 = pd.read_csv("../../Data/ballad_test.csv")
train_ballad2000.head(10)

,가수명,노래제목,발매일,가사,좋아요수,앨범사진,songId,순위
0,WSG워너비 (가야G),그때 그 순간 그대로 (그그그),2022-07-09,잘 지냈지? 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...,88589,https://cdnimg.melon.co.kr/cm2/album/images/11...,35383397,1
1,경서,나의 X에게,2022-04-24,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 ...,105034,https://cdnimg.melon.co.kr/cm2/album/images/10...,34943312,2
2,주호,내가 아니라도,2022-03-25,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던 ...,52955,https://cdnimg.melon.co.kr/cm2/album/images/10...,34787226,4
3,김민석 (멜로망스),취중고백,2021-12-19,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구...,137934,https://cdnimg.melon.co.kr/cm2/album/images/10...,34431086,6
4,임영웅,우리들의 블루스,2022-05-02,잊지는 말아요 함께 했던 날들 눈물이 날 때면 그대 뒤를 돌아보면 돼요 아프지 말...,71880,https://cdnimg.melon.co.kr/cm2/album/images/10...,34908740,7
5,경서예지,다정히 내 이름을 부르면,2021-05-19,끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는...,144360,https://cdnimg.melon.co.kr/cm2/album/images/10...,33496587,8
6,이석훈,바보에게 바보가 (웹툰 '연애의 발견' X 이석훈),2022-07-11,너무 걱정하지는 마 보란 듯이 살아 볼 거야 후회는 사치일 뿐이야 다시 시작해 볼...,16170,https://cdnimg.melon.co.kr/cm2/album/images/11...,35388184,9
7,MSG워너비(M.O.M),듣고 싶을까,2022-02-26,우리 함께 듣던 그 노랠 듣고 싶을까 듣고 나서 잠시 날 생각은 할까 아주 혹시라...,79001,https://cdnimg.melon.co.kr/cm2/album/images/10...,34701627,10
8,케이시 (Kassy),늦은 밤 헤어지긴 너무 아쉬워,2022-05-19,어쩌면 너도 나와 같은 꿈 꿀까 매일 마주 보며 눈뜨는 하루 늦은 밤 헤어지긴 너...,39685,https://cdnimg.melon.co.kr/cm2/album/images/10...,35177030,11
9,임영웅,다시 만날 수 있을까,2022-05-02,너를 위해 해 줄 것이 하나 없어서 보낼 수밖에 없었고 네가 없이 사는 법을 알지...,56804,https://cdnimg.melon.co.kr/cm2/album/images/10...,35008524,12


In [22]:
train_ballad2000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   가수명     1913 non-null   object
 1   노래제목    1913 non-null   object
 2   발매일     1913 non-null   object
 3   가사      1913 non-null   object
 4   좋아요수    1913 non-null   object
 5   앨범사진    1913 non-null   object
 6   songId  1913 non-null   int64 
 7   순위      1913 non-null   int64 
dtypes: int64(2), object(6)
memory usage: 119.7+ KB


In [23]:
train_ballad2000.iloc[1]

가수명                                                      경서
노래제목                                                 나의 X에게
발매일                                              2022-04-24
가사         우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 ...
좋아요수                                                 105034
앨범사진      https://cdnimg.melon.co.kr/cm2/album/images/10...
songId                                             34943312
순위                                                        2
Name: 1, dtype: object

---
### 예시] 30곡을 추출해서 감정분류 후 감정리스트 추가하기

In [24]:
emotion = []
for i in range(0,30):
    pre, scorelist = bf.predict(str(train_ballad2000.iloc[i].가사))
    print("결과 =", pre)
    print(scorelist)
    print('-------------------------------------------')
    emotion.append(pre)

결과 = sad
[('happy', -1305.3548133947374), ('sad', -1241.7277849787345), ('regret', -1242.5056015922573), ('indifference', -1246.1652096128153)]
-------------------------------------------
결과 = sad
[('happy', -969.9209460605761), ('sad', -951.106872908288), ('regret', -960.457777756123), ('indifference', -987.5273266340756)]
-------------------------------------------
결과 = sad
[('happy', -953.9353502524019), ('sad', -916.1658033969543), ('regret', -945.133806131858), ('indifference', -939.4431457407264)]
-------------------------------------------
결과 = regret
[('happy', -1017.6430501968214), ('sad', -1008.5420907852209), ('regret', -1000.4058142783465), ('indifference', -1019.4423648037399)]
-------------------------------------------
결과 = indifference
[('happy', -804.0016671026881), ('sad', -801.3695813965038), ('regret', -779.2270858577328), ('indifference', -769.7012877248226)]
-------------------------------------------
결과 = happy
[('happy', -1368.7291696250047), ('sad', -1410.99276

In [25]:
emotion[1]

'sad'

---
### 1913곡을 추출해서 베이즈 정리 모델에 학습시키고 감정분류 후 감정리스트 추가하기

> 30곡씩 가사를 추출하여 bf에 돌려 pre 값을 emotion List에 합치며  
> 그 가사와 pre값을 다시 bf에 합쳐서 학습시킨다.

In [30]:
range(1913//30+1)

range(0, 64)

In [31]:
emotion = [] # 감정 리스트 선언
for j in range(1913//30 + 1): # 범위 0~64번 돌게 범위 지정
    for i in range(j*30, j*30+30): # 30곡씩 학습 시켜야됨
        if (i == 1913): # 마지막 1913곡일때 for문을 빠져나오게 함.
            break
        pre, scorelist = bf.predict(str(train_ballad2000.iloc[i].가사))
        emotion.append(pre)
        bf.fit(str(train_ballad2000.iloc[i].가사),pre)

In [32]:
len(emotion)

1913

In [33]:
emotion

['sad',
 'sad',
 'sad',
 'regret',
 'indifference',
 'happy',
 'sad',
 'regret',
 'sad',
 'regret',
 'sad',
 'happy',
 'regret',
 'sad',
 'regret',
 'regret',
 'happy',
 'regret',
 'sad',
 'sad',
 'indifference',
 'indifference',
 'happy',
 'indifference',
 'happy',
 'regret',
 'sad',
 'sad',
 'indifference',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'regret',
 'sad',
 'sad',
 'happy',
 'regret',
 'sad',
 'sad',
 'happy',
 'happy',
 'happy',
 'sad',
 'sad',
 'happy',
 'regret',
 'regret',
 'indifference',
 'regret',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'happy',
 'regret',
 'sad',
 'sad',
 'happy',
 'indifference',
 'sad',
 'sad',
 'indifference',
 'sad',
 'sad',
 'happy',
 'sad',
 'sad',
 'indifference',
 'regret',
 'sad',
 'sad',
 'indifference',
 'indifference',
 'happy',
 'happy',
 'sad',
 'sad',
 'happy',
 'sad',
 'indifference',
 'regret',
 'sad',
 'sad',
 'regret',
 'regret',
 'sad',
 'regret',
 'regret',
 'sad',
 'sad',
 'regret',
 'sad',
 'sad',
 'sad',
 'sad',
 'ind

---
### emotion 리스트를 Naive Bayes_ballad2000에 추가하기

In [34]:
Bayes_ballad2000 = pd.read_csv("../../Data/ballad_test.csv")

In [35]:
Bayes_ballad2000

,가수명,노래제목,발매일,가사,좋아요수,앨범사진,songId,순위
0,WSG워너비 (가야G),그때 그 순간 그대로 (그그그),2022-07-09,잘 지냈지? 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...,88589,https://cdnimg.melon.co.kr/cm2/album/images/11...,35383397,1
1,경서,나의 X에게,2022-04-24,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 ...,105034,https://cdnimg.melon.co.kr/cm2/album/images/10...,34943312,2
2,주호,내가 아니라도,2022-03-25,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던 ...,52955,https://cdnimg.melon.co.kr/cm2/album/images/10...,34787226,4
3,김민석 (멜로망스),취중고백,2021-12-19,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구...,137934,https://cdnimg.melon.co.kr/cm2/album/images/10...,34431086,6
4,임영웅,우리들의 블루스,2022-05-02,잊지는 말아요 함께 했던 날들 눈물이 날 때면 그대 뒤를 돌아보면 돼요 아프지 말...,71880,https://cdnimg.melon.co.kr/cm2/album/images/10...,34908740,7
...,...,...,...,...,...,...,...,...
1908,터보,회상 (December),1997-01-01,겨울 오면은 우리 둘이서 항상 왔었던 바닷가 시린 바람과 하얀 파도는 예전 그대로였...,12742,https://cdnimg.melon.co.kr/cm/album/images/003...,1606413,1996
1909,이석훈,고백,2011-06-02,좋은 사람 만나 행복해야 해 너는 웃고 있지만 나는 웃을 수가 없는걸 그래 난 알고...,18569,https://cdnimg.melon.co.kr/cm/album/images/012...,3426230,1997
1910,다비치,나의 첫사랑,2022-05-16,그때의 나는 너를 만나서 하나부터 열까지 모든 순간들이 소중했어 내 첫사랑 잊고 싶...,925,https://cdnimg.melon.co.kr/cm2/album/images/10...,35145139,1998
1911,미스에스,이 나이 먹고 뭐했길래 (Feat. 정슬기),2010-09-10,다시 처음으로 돌아갈 수 있다면 아마 너를 사랑하는 일은 안했을텐데 그 날밤 그 카...,10416,https://cdnimg.melon.co.kr/cm/album/images/010...,2933662,1999


In [36]:
감정 = pd.DataFrame(emotion)
감정

,0
0,sad
1,sad
2,sad
3,regret
4,indifference
...,...
1908,sad
1909,sad
1910,sad
1911,sad


In [37]:
Bayes_ballad2000 = pd.concat([Bayes_ballad2000,감정], axis=1)

In [38]:
Bayes_ballad2000

,가수명,노래제목,발매일,가사,좋아요수,앨범사진,songId,순위,0
0,WSG워너비 (가야G),그때 그 순간 그대로 (그그그),2022-07-09,잘 지냈지? 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...,88589,https://cdnimg.melon.co.kr/cm2/album/images/11...,35383397,1,sad
1,경서,나의 X에게,2022-04-24,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 ...,105034,https://cdnimg.melon.co.kr/cm2/album/images/10...,34943312,2,sad
2,주호,내가 아니라도,2022-03-25,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던 ...,52955,https://cdnimg.melon.co.kr/cm2/album/images/10...,34787226,4,sad
3,김민석 (멜로망스),취중고백,2021-12-19,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구...,137934,https://cdnimg.melon.co.kr/cm2/album/images/10...,34431086,6,regret
4,임영웅,우리들의 블루스,2022-05-02,잊지는 말아요 함께 했던 날들 눈물이 날 때면 그대 뒤를 돌아보면 돼요 아프지 말...,71880,https://cdnimg.melon.co.kr/cm2/album/images/10...,34908740,7,indifference
...,...,...,...,...,...,...,...,...,...
1908,터보,회상 (December),1997-01-01,겨울 오면은 우리 둘이서 항상 왔었던 바닷가 시린 바람과 하얀 파도는 예전 그대로였...,12742,https://cdnimg.melon.co.kr/cm/album/images/003...,1606413,1996,sad
1909,이석훈,고백,2011-06-02,좋은 사람 만나 행복해야 해 너는 웃고 있지만 나는 웃을 수가 없는걸 그래 난 알고...,18569,https://cdnimg.melon.co.kr/cm/album/images/012...,3426230,1997,sad
1910,다비치,나의 첫사랑,2022-05-16,그때의 나는 너를 만나서 하나부터 열까지 모든 순간들이 소중했어 내 첫사랑 잊고 싶...,925,https://cdnimg.melon.co.kr/cm2/album/images/10...,35145139,1998,sad
1911,미스에스,이 나이 먹고 뭐했길래 (Feat. 정슬기),2010-09-10,다시 처음으로 돌아갈 수 있다면 아마 너를 사랑하는 일은 안했을텐데 그 날밤 그 카...,10416,https://cdnimg.melon.co.kr/cm/album/images/010...,2933662,1999,sad


---
### 감정 컬럼 만들기

In [39]:
Bayes_ballad2000.rename(columns={0:'감정'},inplace=True)

In [40]:
Bayes_ballad2000

,가수명,노래제목,발매일,가사,좋아요수,앨범사진,songId,순위,감정
0,WSG워너비 (가야G),그때 그 순간 그대로 (그그그),2022-07-09,잘 지냈지? 조금은 어색해 요즘 좋아 보여 인사 나누며 사실 궁금한 게 너무 많았...,88589,https://cdnimg.melon.co.kr/cm2/album/images/11...,35383397,1,sad
1,경서,나의 X에게,2022-04-24,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 ...,105034,https://cdnimg.melon.co.kr/cm2/album/images/10...,34943312,2,sad
2,주호,내가 아니라도,2022-03-25,사랑이었다 별거 없던 내 하루에 빛이 돼준 단한 사람 나보다 나를 더 아껴 주던 ...,52955,https://cdnimg.melon.co.kr/cm2/album/images/10...,34787226,4,sad
3,김민석 (멜로망스),취중고백,2021-12-19,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구...,137934,https://cdnimg.melon.co.kr/cm2/album/images/10...,34431086,6,regret
4,임영웅,우리들의 블루스,2022-05-02,잊지는 말아요 함께 했던 날들 눈물이 날 때면 그대 뒤를 돌아보면 돼요 아프지 말...,71880,https://cdnimg.melon.co.kr/cm2/album/images/10...,34908740,7,indifference
...,...,...,...,...,...,...,...,...,...
1908,터보,회상 (December),1997-01-01,겨울 오면은 우리 둘이서 항상 왔었던 바닷가 시린 바람과 하얀 파도는 예전 그대로였...,12742,https://cdnimg.melon.co.kr/cm/album/images/003...,1606413,1996,sad
1909,이석훈,고백,2011-06-02,좋은 사람 만나 행복해야 해 너는 웃고 있지만 나는 웃을 수가 없는걸 그래 난 알고...,18569,https://cdnimg.melon.co.kr/cm/album/images/012...,3426230,1997,sad
1910,다비치,나의 첫사랑,2022-05-16,그때의 나는 너를 만나서 하나부터 열까지 모든 순간들이 소중했어 내 첫사랑 잊고 싶...,925,https://cdnimg.melon.co.kr/cm2/album/images/10...,35145139,1998,sad
1911,미스에스,이 나이 먹고 뭐했길래 (Feat. 정슬기),2010-09-10,다시 처음으로 돌아갈 수 있다면 아마 너를 사랑하는 일은 안했을텐데 그 날밤 그 카...,10416,https://cdnimg.melon.co.kr/cm/album/images/010...,2933662,1999,sad


---
### csv 저장하기

In [41]:
Bayes_ballad2000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   가수명     1913 non-null   object
 1   노래제목    1913 non-null   object
 2   발매일     1913 non-null   object
 3   가사      1913 non-null   object
 4   좋아요수    1913 non-null   object
 5   앨범사진    1913 non-null   object
 6   songId  1913 non-null   int64 
 7   순위      1913 non-null   int64 
 8   감정      1913 non-null   object
dtypes: int64(2), object(7)
memory usage: 134.6+ KB


In [42]:
Bayes_ballad2000.to_csv('../../Data/Bayes_ballad2000.csv', index=None)

In [43]:
pd.read_csv("../../Data/Bayes_ballad2000.csv")['감정'].value_counts()

sad             1457
happy            251
regret           123
indifference      82
Name: 감정, dtype: int64

In [44]:
1457 + 251 + 123 + 82

1913